In [3]:
!pip install numpy
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [10]:
import os
import json
import numpy as np

# train_dir = 'D:/signData'
train_dir = '../data/signData'
output_dir = f"{train_dir}/nptxt"
json_folder_path = f'{train_dir}/train/label/landmark'
morpheme_path = f'{train_dir}/train/label/morpheme'


for person in os.listdir(json_folder_path):
    person_output_path = os.path.join(output_dir, str(int(person)))
    os.makedirs(person_output_path, exist_ok=True)
    count=1

    for word in os.listdir(os.path.join(morpheme_path, person)):
        if "F" in word:
            file_path = os.path.join(morpheme_path, person, word)
            with open(file_path, 'r') as morpheme_file:
                data = json.load(morpheme_file)
                data['data'][0]['attributes'][0]['name']

    for word in os.listdir(os.path.join(json_folder_path, person)):
        if "F" in word:
            wordCoordL = np.empty((0, 4, 5, 3))  
            wordCoordR = np.empty((0, 4, 5, 3))
            wordCoordP = np.empty((0, 3, 10))
            print(f"word:{count}")    
            for frame in os.listdir(os.path.join(json_folder_path, person, word)):
                file_path = os.path.join(json_folder_path, person, word, frame)
                try:
                    with open(file_path, 'r') as json_file:
                        data = json.load(json_file)
                        lh_points = data['people']['hand_left_keypoints_3d']
                        rh_points = data['people']['hand_right_keypoints_3d']
                        p_points = data['people']['pose_keypoints_3d']

                        preFrameCoordP = np.array([[(960 * p_points[i] + 960 - 420) / (1500 - 420), 
                                                    (1080 * p_points[i + 1] + 540) / 1080,
                                                    (p_points[32 + 2] - p_points[i + 2]) / 10]
                                                   for i in range(0, len(p_points), 4)], dtype=np.float32)

                        preFrameCoordL = np.array([[(960 * lh_points[i] + 960 - 420) / (1500 - 420),
                                                    (1080 * lh_points[i + 1] + 540) / 1080,
                                                    (lh_points[2] - lh_points[i + 2]) / 10]
                                                   for i in range(4, len(lh_points), 4)], dtype=np.float32)

                        preFrameCoordR = np.array([[(960 * rh_points[i] + 960 - 420) / (1500 - 420),
                                                    (1080 * rh_points[i + 1] + 540) / 1080,
                                                    (rh_points[2] - rh_points[i + 2]) / 10]
                                                   for i in range(4, len(rh_points), 4)], dtype=np.float32)

                        preFrameCoordP = preFrameCoordP[[i for i in range(19) if (0 <= i <= 7) or (17 <= i <= 18)]]

                        frameCoordL=preFrameCoordL.reshape(5,4,3).transpose(1,0,2)[::-1]
                        frameCoordR=preFrameCoordR.reshape(5,4,3).transpose(1,0,2)[::-1]
                        frameCoordP=preFrameCoordP.T

                except json.JSONDecodeError as e:
                    print(f"Error reading {file_path}: {e}")
                    break

            wordCoordL = np.append(wordCoordL, [frameCoordL], axis=0)
            wordCoordR = np.append(wordCoordR, [frameCoordR], axis=0)
            wordCoordP = np.append(wordCoordP, [frameCoordP], axis=0)
            label=[count,int(person)]
            word_output_path = os.path.join(person_output_path, f'{label[0]}.npz')
            np.savez(word_output_path, wordCoordL=wordCoordL, wordCoordR=wordCoordR, wordCoordP=wordCoordP, label=label)
            print(f"Saved {word_output_path}")
            count+=1
        

            
            
            
def load_data(person,word):
    path=f"{output_dir}/{person}/{word}.npz"
    data = np.load(path)
    wordCoordL = data['wordCoordL']
    wordCoordR = data['wordCoordR']
    wordCoordP = data['wordCoordP']
    label = data['label']

    return wordCoordL, wordCoordR, wordCoordP, label


def load_word(person, start, num):
    words = []
    for WNum in range(start, start + num):
        wordCoordL, wordCoordR, wordCoordP, label = load_data(person, WNum)
        words.append([wordCoordL, wordCoordR, wordCoordP, label])
    return words



#todo
#아래 코드를 함수화 , 포문돌려서 한번에 다 불러오기
#함수에 프레임 갯수(영상길이) 호출 가능하게 구현
#한 단어 내에서 호출할떄 6개의 프레임씩 겹쳐서 호출 하는 기능 구현




고민
뻔뻔
수어
남아
눈
독신
음료수
발가락
슬프다
자극
어색하다
여아
안타깝다
학교연혁
외국인
영아
신사
뉴질랜드
나사렛대학교
알아서
장애인
열아홉번째
침착
성실
싫어하다
급하다
필기시험
병문안
검사
결승전
낚시터
낚시대
당뇨병
독서
매표소
면역
배드민턴
변비
병명
보건소
불면증
불행
붕대
사위
설사
성병
방충
소화제
손자
수면제
수집가
여행지
예식장
올림픽경기
회복
첫번째
운동경기
입원
재혼
진단서
치료
치료법
감기
손녀
친아들
퇴원
한약
한약방
빈혈
화상
가래떡
고깃국
고추
고추가루
배추국
꽈베기
벌꿀
꿀물
냄비
찬물
다과
지방경찰청장
된장찌게
돼지고기
두부
딸기
라면
막걸리
무
밥그릇
밥솥
보신탕
부엌
사골
떡국
소불고기
비빔밥
사과
사이다
달다
술잔
식당
쌀가루
냉커피
양식
위스키
술
용품
유리그릇
짬뽕
참외
칼국수
콜라
콩가루
통조림
와인
백설기
가죽신
군복
포켓
양산
운동화
잠옷
이불
소주
커피
행거
월세
전세
치약
가수
체감
경고
경찰서
공식
과장
관광버스
교수
찻길
독일어
반환
음주운전
동아리
디자이너
마을버스
구급차
개입하지않는다
음주운전
사무
신입학
여행사
기차역
예술가
외국어
요리사
에어컨
교장
여객선
강요
운동선수
퇴임
의사
이력서
자가용
재택근무
주차장
채팅
출근
코치
터미널
근무
통역사
퇴사
사직
판결
평가
두번째
교통안전교육
화상채팅
회식
교통기본교육
후원
희생
간호사
교통법교육
계약금
고용
연령제한
항공사
축구장
예절
사업장
버스값
논
대기업
사육장
백화점
압류
거지
수입
신용카드
백수
아르바이트
샛길
일시불
집주인
한국은행
경찰청장
가정교육
고등학교
고등학생
교무실
개학
교실
교육자
국어학
낙제
독서실
등교
구직
주유소
맹학교
복습
분필
상장
서점
학업
실습
여학교
여학생
연구소
예습
유학
의학
지도
재학
연구
학부모
학식
훈련
경기도
경기도
경북
울릉도
울산
전남
청주
충북
충남
불경
연구실
남산
불경
성경책
신학
장로교
일요일
회개
소감
장난
장인
희생자
디자인
돌무덤
쇠기둥
동화책
목공예
문학
문화유산
예술
초상화
토기
흑

KeyboardInterrupt: 

In [1]:
import numpy as np
test_hand=np.empty((0, 3)) 
test_pose=np.empty((0, 3)) 

for i in range(0,21):
    arr=np.array([i,i,i])
    if i != 0:
        test_hand=np.append(test_hand,[arr],axis=0)
    if (0<=i<=7) or (17<=i<=18):
        test_pose=np.append(test_pose,[arr],axis=0)


test_hand=test_hand.reshape(5,4,3).transpose(1,0,2)[::-1]
test_pose=test_pose.T



print(test_hand)
print(test_pose)

[[[ 4.  4.  4.]
  [ 8.  8.  8.]
  [12. 12. 12.]
  [16. 16. 16.]
  [20. 20. 20.]]

 [[ 3.  3.  3.]
  [ 7.  7.  7.]
  [11. 11. 11.]
  [15. 15. 15.]
  [19. 19. 19.]]

 [[ 2.  2.  2.]
  [ 6.  6.  6.]
  [10. 10. 10.]
  [14. 14. 14.]
  [18. 18. 18.]]

 [[ 1.  1.  1.]
  [ 5.  5.  5.]
  [ 9.  9.  9.]
  [13. 13. 13.]
  [17. 17. 17.]]]
[[ 0.  1.  2.  3.  4.  5.  6.  7. 17. 18.]
 [ 0.  1.  2.  3.  4.  5.  6.  7. 17. 18.]
 [ 0.  1.  2.  3.  4.  5.  6.  7. 17. 18.]]


In [3]:
load_data(1,1)=
print(data)
wordCoordL = data['wordCoordL']
wordCoordR = data['wordCoordR']
wordCoordP = data['wordCoordP']
label = data['label']
person = data['person']

print(f"Left Hand Shape: {wordCoordL.shape}")
print(wordCoordL)
print(f"Right Hand Shape: {wordCoordR.shape}")
print(wordCoordR)
print(f"Pose Shape: {wordCoordP.shape}")
print(wordCoordP)
print(f"Label Shape: {label.shape}")
print(label)
print(f"Person Shape: {person.shape}")
print(person)

NpzFile '../data/signData/nptxt/1/1.npz' with keys: wordCoordL, wordCoordR, wordCoordP, label


KeyError: 'person is not a file in the archive'

In [10]:
import os
import json
import numpy as np

# train_dir = 'D:/signData'
train_dir = '../data/signData'
output_dir = f"{train_dir}/nptxt"
json_folder_path = f'{train_dir}/train/label/landmark'

def load_data(person,word):
    path=f"{output_dir}/{person}/{word}.npz"
    data = np.load(path)
    wordCoordL = data['wordCoordL']
    wordCoordR = data['wordCoordR']
    wordCoordP = data['wordCoordP']
    label = data['label']

    return [wordCoordL, wordCoordR, wordCoordP], label


def load_word(person, start, num):
    words = []
    count = 0
    for WNum in range(start, start + num):
        word, label = load_data(person, WNum)
        words.append([word, label])
        count += 1
    return words, count


test_data, count = load_word(1, 1, 10)
print(test_data[3][0][0])



[[[[ 6.14022672e-01  9.19334590e-01  3.75703815e-03]
   [ 6.20938718e-01  9.58689809e-01  2.65272870e-03]
   [ 6.28104150e-01  9.61659551e-01  1.00853143e-03]
   [ 6.39128447e-01  9.57970023e-01 -2.94943689e-04]
   [ 6.53091311e-01  9.41438258e-01 -7.00237579e-04]]

  [[ 6.19099796e-01  8.92214656e-01  3.19638848e-03]
   [ 6.28276467e-01  9.44320500e-01  3.78554873e-03]
   [ 6.39373422e-01  9.48753953e-01  2.22581322e-03]
   [ 6.49575651e-01  9.43401873e-01  7.56198191e-04]
   [ 6.60642564e-01  9.31000590e-01 -1.02631857e-04]]

  [[ 6.22536898e-01  8.56616139e-01  2.69892020e-03]
   [ 6.40385211e-01  9.22769904e-01  4.63342993e-03]
   [ 6.52831316e-01  9.27452147e-01  3.45962355e-03]
   [ 6.64397597e-01  9.24696088e-01  1.47253496e-03]
   [ 6.71019018e-01  9.18476284e-01 -8.81781234e-05]]

  [[ 6.41746342e-01  8.26315999e-01  1.15207746e-03]
   [ 6.51965678e-01  8.83493364e-01  3.73803405e-03]
   [ 6.67385399e-01  8.91724467e-01  1.96974608e-03]
   [ 6.76198959e-01  8.91030788e-01  6.9

In [21]:
test = []
for word in os.listdir(os.path.join(morpheme_path, "01")):
        if "F" in word:
            file_path = os.path.join(morpheme_path, "01", word)
            with open(file_path, 'r') as morpheme_file:
                data = json.load(morpheme_file)
                test.append(data['data'][0]['attributes'][0]['name'])
                print(data['data'][0]['attributes'][0]['name'])

고민
뻔뻔
수어
남아
눈
독신
음료수
발가락
슬프다
자극
어색하다
여아
안타깝다
학교연혁
외국인
영아
신사
뉴질랜드
나사렛대학교
알아서
장애인
열아홉번째
침착
성실
싫어하다
급하다
필기시험
병문안
검사
결승전
낚시터
낚시대
당뇨병
독서
매표소
면역
배드민턴
변비
병명
보건소
불면증
불행
붕대
사위
설사
성병
방충
소화제
손자
수면제
수집가
여행지
예식장
올림픽경기
회복
첫번째
운동경기
입원
재혼
진단서
치료
치료법
감기
손녀
친아들
퇴원
한약
한약방
빈혈
화상
가래떡
고깃국
고추
고추가루
배추국
꽈베기
벌꿀
꿀물
냄비
찬물
다과
지방경찰청장
된장찌게
돼지고기
두부
딸기
라면
막걸리
무
밥그릇
밥솥
보신탕
부엌
사골
떡국
소불고기
비빔밥
사과
사이다
달다
술잔
식당
쌀가루
냉커피
양식
위스키
술
용품
유리그릇
짬뽕
참외
칼국수
콜라
콩가루
통조림
와인
백설기
가죽신
군복
포켓
양산
운동화
잠옷
이불
소주
커피
행거
월세
전세
치약
가수
체감
경고
경찰서
공식
과장
관광버스
교수
찻길
독일어
반환
음주운전
동아리
디자이너
마을버스
구급차
개입하지않는다
음주운전
사무
신입학
여행사
기차역
예술가
외국어
요리사
에어컨
교장
여객선
강요
운동선수
퇴임
의사
이력서
자가용
재택근무
주차장
채팅
출근
코치
터미널
근무
통역사
퇴사
사직
판결
평가
두번째
교통안전교육
화상채팅
회식
교통기본교육
후원
희생
간호사
교통법교육
계약금
고용
연령제한
항공사
축구장
예절
사업장
버스값
논
대기업
사육장
백화점
압류
거지
수입
신용카드
백수
아르바이트
샛길
일시불
집주인
한국은행
경찰청장
가정교육
고등학교
고등학생
교무실
개학
교실
교육자
국어학
낙제
독서실
등교
구직
주유소
맹학교
복습
분필
상장
서점
학업
실습
여학교
여학생
연구소
예습
유학
의학
지도
재학
연구
학부모
학식
훈련
경기도
경기도
경북
울릉도
울산
전남
청주
충북
충남
불경
연구실
남산
불경
성경책
신학
장로교
일요일
회개
소감
장난
장인
희생자
디자인
돌무덤
쇠기둥
동화책
목공예
문학
문화유산
예술
초상화
토기
흑

In [26]:
test3 = []
count = 0
for i in range(1, 17):
    for word in os.listdir(os.path.join(morpheme_path, str(i).zfill(2))):
            if "F" in word:
                file_path = os.path.join(morpheme_path, str(i).zfill(2), word)
                with open(file_path, 'r') as morpheme_file:
                    data = json.load(morpheme_file)
                    try:
                        data['data'][0]['attributes'][0]['name']

                    except IndexError as e:
                        print(f"Error reading {file_path}: {e}")
                        continue

Error reading ../data/signData/train/label/morpheme/02/NIA_SL_WORD1056_REAL25_F_morpheme.json: list index out of range
Error reading ../data/signData/train/label/morpheme/02/NIA_SL_WORD2742_REAL02_F_morpheme.json: list index out of range
Error reading ../data/signData/train/label/morpheme/04/NIA_SL_WORD0796_REAL04_F_morpheme.json: list index out of range
Error reading ../data/signData/train/label/morpheme/09/NIA_SL_WORD1082_REAL09_F_morpheme.json: list index out of range
Error reading ../data/signData/train/label/morpheme/09/NIA_SL_WORD1612_REAL09_F_morpheme.json: list index out of range
Error reading ../data/signData/train/label/morpheme/09/NIA_SL_WORD1613_REAL09_F_morpheme.json: list index out of range
Error reading ../data/signData/train/label/morpheme/09/NIA_SL_WORD1614_REAL09_F_morpheme.json: list index out of range
Error reading ../data/signData/train/label/morpheme/09/NIA_SL_WORD1615_REAL09_F_morpheme.json: list index out of range
Error reading ../data/signData/train/label/morph